In [13]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, concatenate
from tensorflow.keras.activations import relu
from DataLoader import DataLoader
from halfkp import get_halfkp_indeicies
import chess
from chess import Board
import matplotlib.pyplot as plt

import numpy as np

In [10]:
training_generator = DataLoader(64, "train")
val_generator = DataLoader(64, "val", shuffle=False)

In [11]:
lossHistory = []
valHistory = []
class NnueCallbacks(tf.keras.callbacks.Callback):
  def on_batch_end(self, batch, logs=None):
    lossHistory.append(logs['loss'])
    if batch and batch % 1000 == 0:
      pred = self.model.predict(val_generator)
      loss = (pred.flatten() - val_generator.labels.flatten()[0:len(val_generator)*64])**2
      loss = np.sum(loss) / len(loss)
      valHistory.append(loss)

      plt.plot(valHistory)
      plt.savefig("val.png", dpi=200)
      plt.clf()

      plt.plot(lossHistory[20:])
      plt.savefig("loss.png", dpi=200)
      plt.clf()
      
      print(" val_loss : {}".format(loss))

      self.model.save("models/model-{}-{}".format(len(valHistory), int(loss*10000)))

In [12]:
from keras import backend as K
def clipped_relu(x):
    return K.relu(x, max_value=2)

In [15]:
accumulator = Dense(256, clipped_relu)
input1 = Input((64*64*12,))
input2 = Input((64*64*12,))
layers = concatenate([accumulator(input1), accumulator(input2)])
layers = Dense(32, clipped_relu)(layers)
layers = Dense(32, clipped_relu)(layers)
layers = Dense(1)(layers)
model = Model(inputs=[input1, input2], outputs=layers)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse', metrics=['mae'])

In [ ]:
model = tf.keras.models.load_model("production/")

In [16]:
model.fit(training_generator, validation_data=val_generator, epochs=1, callbacks=[NnueCallbacks()])

  999/62500 [..............................] - ETA: 22:44 - loss: 13.9054 - mae: 2.3457 val_loss : 13.14160850172076


2022-04-12 10:43:15.255170: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/model-1-131416/assets
 2000/62500 [..............................] - ETA: 20:56 - loss: 11.9332 - mae: 2.1862 val_loss : 10.41320055312828
INFO:tensorflow:Assets written to: models/model-2-104132/assets
 2997/62500 [>.............................] - ETA: 18:50 - loss: 10.8010 - mae: 2.0904 val_loss : 9.268189010120038
INFO:tensorflow:Assets written to: models/model-3-92681/assets
 3999/62500 [>.............................] - ETA: 17:51 - loss: 10.0525 - mae: 2.0255 val_loss : 8.527112578538679
INFO:tensorflow:Assets written to: models/model-4-85271/assets
 4998/62500 [=>............................] - ETA: 17:00 - loss: 9.5031 - mae: 1.9746 val_loss : 8.065849491198268
INFO:tensorflow:Assets written to: models/model-5-80658/assets
 6000/62500 [=>............................] - ETA: 16:31 - loss: 9.0568 - mae: 1.9319 val_loss : 7.964719078336325
INFO:tensorflow:Assets written to: models/model-6-79647/assets
 7000/62500 [==>.....................

In [ ]:
indicies = get_halfkp_indeicies(Board("r3k1nr/pp2ppbp/n2p2p1/q1p5/3N1P2/1P2P3/PBPP3P/RN1K1B1R w kq - 1 9"))
indicies = indicies.reshape(2, -1)
indicies = (np.array([indicies[0], indicies[0]]), np.array([indicies[1], indicies[1]]))
pred = model.predict(indicies)
print(pred[0])

In [ ]:
pred = model.predict(val_generator[0][0]) * 2 - 1
act = val_generator[0][1] * 2 - 1
for i in range(128):
    print(pred[i], act[i], val_generator.getfen(0)[i])

In [7]:
import time
# model.save("models/model-{}".format(int(time.time())))
model.save("production/")

INFO:tensorflow:Assets written to: production/assets
